Implementation using  TensorFlow


In [54]:
import numpy as np
import pandas as pd

In [56]:
census=pd.read_csv("/Users/apekshagyawali/Desktop/deeplearn/census_data.csv")

In [57]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Convert strings as labes to 0s and 1s using .apply()

In [58]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [59]:
def label_fix(label):
    if label=='<=50K':
        return 0
    else:
        return 1

In [60]:
census['income_bracket']=census['income_bracket'].apply(label_fix)

perform train test split

In [61]:
from sklearn.model_selection import train_test_split 

In [62]:
x_data=census.drop('income_bracket', axis=1)
y_labels=census['income_bracket']
x_train, x_test, y_train, y_test=train_test_split(x_data, y_labels, test_size=0.3, random_state=101)

In [63]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [64]:
import tensorflow as tf

create features columns for the categorical values

In [65]:
gender=tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation=tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status=tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship=tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education=tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass=tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country=tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


create feature columns for continuous using numeric_columns

In [66]:
age=tf.feature_column.numeric_column("age")
education_num=tf.feature_column.numeric_column("education_num")
capital_gain=tf.feature_column.numeric_column("capital_gain")
capital_loss= tf.feature_column.numeric_column("capital_loss")
hours_per_week=tf.feature_column.numeric_column("hours_per_week")






Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


put all these variables into a single list 

In [67]:
feat_cols=[gender,occupation, marital_status, relationship, education, workclass, native_country, age, education_num, capital_gain, capital_loss, hours_per_week]

In [69]:
# Create input functions using tf.data.Dataset API
def input_func(features, labels, batch_size=100, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(features))
    dataset = dataset.batch(batch_size)
    return dataset

In [80]:
# Define input function for prediction
def predict_input_fn(features, batch_size=100):
    dataset = tf.data.Dataset.from_tensor_slices(dict(features))
    dataset = dataset.batch(batch_size)
    return dataset

In [81]:
# Create input functions for training and evaluation
train_input_fn = lambda: input_func(x_train, y_train)
eval_input_fn = lambda: input_func(x_test, y_test, shuffle=False)


In [82]:
model=tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/5q/xycqttws0ll8pjcw509y6tmh0000gp/T/tmpa9bjstla', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [83]:
model.train(input_fn=train_input_fn, steps=5000)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/5q/xycqttws0ll8pjcw509y6tmh0000gp/T/tmpa9bjstla/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...


2024-02-18 15:52:44.261700: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int64 and shape [22792]
	 [[{{node Placeholder/_2}}]]
2024-02-18 15:52:44.262462: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype string and shape [22792]
	 [[{{node Placeholder/_10}}]]


INFO:tensorflow:loss = 0.6931472, step = 0
INFO:tensorflow:global_step/sec: 216.457
INFO:tensorflow:loss = 2.7252472e-06, step = 100 (0.463 sec)
INFO:tensorflow:global_step/sec: 700.708
INFO:tensorflow:loss = 2.427754e-06, step = 200 (0.142 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 228...
INFO:tensorflow:Saving checkpoints for 228 into /var/folders/5q/xycqttws0ll8pjcw509y6tmh0000gp/T/tmpa9bjstla/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 228...
INFO:tensorflow:Loss for final step: 1.5268077e-05.


create prediction input function

In [84]:
pred_input_fn = lambda: predict_input_fn(x_test)


In [85]:
predictions=list(model.predict(input_fn=pred_input_fn))

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/5q/xycqttws0ll8pjcw509y6tmh0000gp/T/tmpa9bjstla/model.ckpt-228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2024-02-18 15:52:57.840665: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype string and shape [9769]
	 [[{{node Placeholder/_8}}]]
